<h1><center><font color='Black'>Crawling</font></center></h1>





In [ ]:
#---------Imports & Installation---------#
#pip install selenium
#pip install webdriver
#pip install lxml
#pip install wordcloud
#pip install seaborn
#pip install matplotlib
#pip install python-bidi
#pip install sklearn



import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
import requests
import json
from datetime import date
import datetime as dt
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


todays_date = date.today() #Get Today'S Date
curryear = todays_date.year
swipe = curryear - 2014 #Start From Curr Year To 2014

we are going to use <u>**Selenium**</u> to navigate automaticly in the website we want to crwaling, until we find the table with all the data we looking for on the site.




In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install()) #install Are Auto-Website
driver.implicitly_wait(10) #Wait until everything is upload
driver.get('https://www.oref.org.il//12481-he/Pakar.aspx')


#Clicks:

element = driver.find_element_by_id("ah-chooseDates").click()#Calendar Buttom
sleep(0.9)
element = driver.find_element_by_id("txtDateFrom").click()#Open Calendar Itself
sleep(0.9)
element = driver.find_element_by_class_name("datepicker-switch").click()#Switch From Days To Month
sleep(0.9)

xp1= "//*[@id='ctl00_Body']/div[6]/div[2]/table/thead/tr/th[1]"
xp2="//*[@id='ctl00_Body']/div[6]/div[2]/table/tbody/tr/td/span[7]"
xp3="//*[@id='ctl00_Body']/div[6]/div[1]/table/tbody/tr[4]/td[5]"

for x in range(swipe): #Select The Range
    element = driver.find_element_by_xpath(xp1).click()
    sleep(0.3)

element = driver.find_element_by_xpath(xp2).click()#Choose The Month
sleep(0.5)
element = driver.find_element_by_xpath(xp3).click()#Choose The Day
sleep(0.5)

Once we have finished navigating the site we are going to use **BeautifulSoup** to Crawling

In [ ]:
sleep(10)#Wait Until All The Data In The Table Will Showing Up
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

soup_date = soup("li",attrs={"class" : "alertTableDateCol"})
soup_time = soup("li",attrs={"class" : "alertTableTimeCol"})
soup_zone = soup("li",attrs={"class" : "alertTableCityNameCol"})


date = [ i.get_text().strip() for i in soup_date ]
time = [ i.get_text().strip() for i in soup_time ]
zone = [ i.get_text().strip() for i in soup_zone ]

d = {"Zone":zone , "Date": date,"Time": time}
df = pd.DataFrame(data=d)#create a data frame with all the data
df.drop(index= df.index[0] , axis=0, inplace=True)
df.drop(index=df.index[-1], axis=0, inplace=True)
df_copy = df.copy()


In [ ]:
df.to_csv("Data/df.csv", encoding='utf-8-sig', index=False)#Create a csv File

The names of the cities do not appear in their original names, but their names appear as the names of their regions.

So we need to crawl to another website, from there we take the names for each region we have.

In [ ]:
url = "https://www.mivzaklive.co.il/%D7%94%D7%AA%D7%A8%D7%90%D7%AA-%D7%A6%D7%91%D7%A2-%D7%90%D7%93%D7%95%D7%9D-%D7%9E%D7%A1%D7%A4%D7%A8%D7%99-%D7%A4%D7%95%D7%9C%D7%99%D7%92%D7%95%D7%A0%D7%99%D7%9D-%D7%95%D7%96%D7%9E%D7%A0%D7%99-%D7%94"
res = requests.get(url)


soup = BeautifulSoup(res.content, 'html.parser')

In [ ]:
soup_zone_time = soup.find_all(attrs={'style':'width: 150px;'})
soup_zone_name = soup.find_all(attrs={'style':'width: 234px;'})
soup_zone_number = soup.find_all(attrs={'style':'width: 148px;'})

zone_name = [ i.get_text().strip() for i in soup_zone_name ]
defense_time = [i.get_text().strip() for i in soup_zone_time] 
zone_number = [i.get_text().strip() for i in soup_zone_number] 

In [ ]:
dic2 = {"Zone Name" : zone_name , "Zone Number": zone_number, "Defense Time" : defense_time }
df2 = pd.DataFrame(data = dic2)#create a second data frame with all the data
df2.drop(index= df2.index[0] , axis=0, inplace=True) #drop the first row


In [ ]:
df

In [ ]:
df2

In [ ]:
df2.to_csv("Data/df2.csv", encoding='utf-8-sig' ,index=False )
df.to_csv("Data/df.csv", encoding='utf-8-sig' , index=False)